<a href="https://colab.research.google.com/github/RifqiHafizuddin/xLSTM-Sentiment-Analisis/blob/main/EndTermTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 8.6 MB/s eta 0:00:00


In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
import string
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
import emoji
nltk.download('punkt')
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from keras.layers import LSTM, Dense, SimpleRNN, Embedding, Flatten, Dropout
from keras.activations import softmax
from sklearn.model_selection import train_test_split
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#read data
df=pd.read_csv('sentiment_tweets3.csv')
df.head()


,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [ ]:
# Rename columns
df.rename(columns={'message to examine': 'Text', 'label (depression result)': 'Label'}, inplace=True)
df.head()

,Index,Text,Label
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [ ]:
df.shape

(10314, 3)

In [ ]:
# Convert 'Text' column to lowercase
df['Text'] = df['Text'].str.lower()
df.head()

,Index,Text,Label
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat need to send 'em to my accountant tomo...,0
4,540,add me on myspace!!! myspace.com/lookthunder,0


# Preprocess

remove HTML

- The function remove_html_tags accepts a string of text as its parameter.
- It creates a BeautifulSoup object soup by passing the input text and specifying the parser as `html.parser`. BeautifulSoup is a Python library for parsing `HTML` and `XML` documents.
- The get_text() method of the BeautifulSoup object is then called to extract the text content from the HTML, excluding any `HTML tags`.
- The cleaned text without `HTML tags` is returned from the function.
After defining the function, the code applies this function to the `Text` column of a DataFrame named df using the `apply()` method. This effectively `removes HTML tags` from each entry in the `Text` column of the DataFrame.

In [ ]:
# Function to remove HTML tags from text
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

# Remove HTML tags from 'Text' column
df['Text'] = df['Text'].apply(remove_html_tags)

In [ ]:
df.head()

,Index,Text,Label
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat need to send 'em to my accountant tomo...,0
4,540,add me on myspace!!! myspace.com/lookthunder,0


In [ ]:
# Define a function to remove URLs using regular expressions
def remove_urls(text):
    return re.sub(r'http\S+|www\S+', '', text)

# Apply the function to the 'Text' column
df['Text'] = df['Text'].apply(remove_urls)

In [ ]:
df.head()

,Index,Text,Label
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga,0
2,220,@comeagainjen -,0
3,288,@lapcat need to send 'em to my accountant tomo...,0
4,540,add me on myspace!!! myspace.com/lookthunder,0


In [ ]:
string.punctuation

# Define the punctuation characters to remove
punctuation = string.punctuation

In [ ]:
# Function to remove punctuation from text
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punctuation))

# Apply remove_punctuation function to 'Text' column
df['Text'] = df['Text'].apply(remove_punctuation)

In [ ]:
df.head()

,Index,Text,Label
0,106,just had a real good moment i missssssssss him...,0
1,217,is reading manga,0
2,220,comeagainjen,0
3,288,lapcat need to send em to my accountant tomorr...,0
4,540,add me on myspace myspacecomlookthunder,0


Handling ChatWords
- The code defines a dictionary named `chat_words` which contains abbreviations commonly used in chat conversations along with their corresponding full forms

- Then, a function named `replace_chat_words` is defined, which takes a text input, splits it into individual words, and iterates through each word. If a word is found in the `chat_words` dictionary (after converting it to lowercase), it replaces the word with its full form. Finally, it joins all the words back together into a single string

- The `replace_chat_words` function is applied to the 'Text' column of a DataFrame named `df` using the `apply` method, which replaces the chat words in each text entry of the 'Text' column

- This code essentially performs text preprocessing by replacing chat abbreviations with their full forms, which can be useful for tasks like sentiment analysis, text classification, or any natural language processing (NLP) task where understanding the full meaning of text is important

In [ ]:
# Define a dictionary of chat word mappings
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "B4N": "Bye For Now",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "ILU: I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don't care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can't stop laughing"
}

In [ ]:
# Function to replace chat words with their full forms
def replace_chat_words(text):
    words = text.split()
    for i, word in enumerate(words):
        if word.lower() in chat_words:
            words[i] = chat_words[word.lower()]
    return ' '.join(words)

# Apply replace_chat_words function to 'Text' column
df['Text'] = df['Text'].apply(replace_chat_words)


In [ ]:
# Download NLTK stopwords corpus
nltk.download('stopwords')

# Get English stopwords from NLTK
stop_words = set(stopwords.words('english'))

# Function to remove stop words from text
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Apply remove_stopwords function to 'Text' column
df['Text'] = df['Text'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Function to remove emojis from text
def remove_emojis(text):
    return emoji.demojize(text)

# Apply remove_emojis function to 'Text' column
df['Text'] = df['Text'].apply(remove_emojis)

In [ ]:
# Intilize Lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# Apply
df['Text_lemmatized'] = df['Text'].apply(lambda x: ' '.join([wordnet_lemmatizer.lemmatize(word , pos='v') for word in x.split()]))

# Head
df.head()

,Index,Text,Label,Text_lemmatized
0,106,real good moment missssssssss much,0,real good moment missssssssss much
1,217,reading manga,0,read manga
2,220,comeagainjen,0,comeagainjen
3,288,lapcat need send em accountant tomorrow oddly ...,0,lapcat need send em accountant tomorrow oddly ...
4,540,add myspace myspacecomlookthunder,0,add myspace myspacecomlookthunder


# Model Input

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Tokenize the text
max_words = 10000
max_len = 128
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['Text'])
X = tokenizer.texts_to_sequences(df['Text'])
X = pad_sequences(X, maxlen=max_len)

# Encode the sentiment labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Label'])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.long)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create DataLoader for batching
batch_size = 32
train_dataset = SentimentDataset(X_train_tensor, y_train_tensor)
test_dataset = SentimentDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Model Building

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define xLSTMCell as before
def binary_accuracy(y_pred, y_true, threshold=0.5):
    # Convert probabilities to binary predictions
    y_pred = (y_pred > threshold).float()
    # Calculate accuracy
    correct = (y_pred == y_true).float().sum()
    accuracy = correct / y_true.shape[0]
    return accuracy
def multiclass_accuracy(y_pred, y_true):
    # Get the predicted class by finding the index with the highest probability
    _, y_pred_classes = torch.max(y_pred, dim=1)

    # Calculate accuracy
    correct = (y_pred_classes == y_true).float().sum()
    accuracy = correct / y_true.shape[0]
    return accuracy

class xLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(xLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Define xLSTM gates
        self.gates = nn.Linear(input_size + hidden_size, 4 * hidden_size)
        self.output_gate = nn.Linear(input_size + hidden_size, hidden_size)

    def forward(self, input, hidden):
        hx, cx = hidden
        gates = self.gates(torch.cat((input, hx), 1))

        # Split gates into individual components
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        # Apply activation functions
        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)

        # Update cell state
        cy = (forgetgate * cx) + (ingate * cellgate)

        # Compute output
        hy = outgate * torch.tanh(cy)

        return hy, cy

class xLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(xLSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.cells = nn.ModuleList([xLSTMCell(input_size, hidden_size)])
        self.cells.extend([xLSTMCell(hidden_size, hidden_size) for _ in range(num_layers - 1)])

    def forward(self, input, hidden=None):
        batch_size, seq_len, _ = input.size()

        if hidden is None:
            hidden = self.init_hidden(batch_size)

        outputs = []
        for t in range(seq_len):
            x = input[:, t, :]
            for layer in range(self.num_layers):
                hx, cx = hidden[layer]
                x, cx = self.cells[layer](x, (hx, cx))
                hidden[layer] = (x, cx)
            outputs.append(x)

        return torch.stack(outputs, dim=1), hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        return [(weight.new(batch_size, self.hidden_size).zero_(),
                weight.new(batch_size, self.hidden_size).zero_())
                for _ in range(self.num_layers)]

# Define model, loss function, and optimizer

class SentimentAnalyzer(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers, num_classes):
        super(SentimentAnalyzer, self).__init__()
        # Use standard embedding instead of BERT
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.xlstm = xLSTM(embed_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, num_classes)  # Change here for multiclass

    def forward(self, x):
        x = self.embedding(x)  # Convert word indices to embeddings
        x, _ = self.xlstm(x)
        x = self.fc(x[:, -1, :])  # Use last output for classification
        return x  # Output raw logits for CrossEntropyLoss

# Set parameters
vocab_size = 10000  # Adjust this based on your vocabulary size
embed_size = 768  # Common embedding size (e.g., 300 for word2vec)
hidden_size = 128
num_layers = 2
num_classes = 2  # Adjust based on your dataset (e.g., binary or multiclass classification)

# Initialize the model, criterion, and optimizer
model = SentimentAnalyzer(vocab_size=vocab_size, embed_size=embed_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()  # Suitable for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training and evaluation logic remains the same
num_epochs = 5  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        targets = targets.long()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)  # No need to squeeze() for CrossEntropyLoss
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_loss/len(train_loader)}')

    # Evaluate on the test set
    model.eval()
    test_loss = 0.0
    test_accuracy = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            targets = targets.long()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            # Calculate accuracy
            accuracy = multiclass_accuracy(outputs, targets)
            test_accuracy += accuracy.item()

    avg_test_loss = test_loss / len(test_loader)
    avg_test_accuracy = test_accuracy / len(test_loader)
    print(f'Test Loss: {avg_test_loss}, Test Accuracy: {avg_test_accuracy * 100:.2f}%')


Epoch 1/5, Loss: 0.08159913945683213
Test Loss: 0.04625381943411552, Test Accuracy: 99.04%
Epoch 2/5, Loss: 0.030240662397082188
Test Loss: 0.04172996485140175, Test Accuracy: 99.09%
Epoch 3/5, Loss: 0.012055616433340325
Test Loss: 0.05198847277519795, Test Accuracy: 99.23%
Epoch 4/5, Loss: 0.003422551471568261
Test Loss: 0.07033772372056914, Test Accuracy: 99.18%
Epoch 5/5, Loss: 0.00154927637309677
Test Loss: 0.08141411243135595, Test Accuracy: 99.23%


# Analisis Hasil

In [ ]:
y_pred=[]
y_true=[]
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X, batch_y
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())  # Ensure predictions are on CPU and converted to numpy
        y_true.extend(batch_y.cpu().numpy())   # Ensure true labels are on CPU and converted to numpy

# Generate classification report
report = classification_report(y_true, y_pred, target_names=['Negative', 'Positive'])
print(report)

              precision    recall  f1-score   support

    Negative       0.99      1.00      1.00      1614
    Positive       1.00      0.97      0.98       449

    accuracy                           0.99      2063
   macro avg       0.99      0.98      0.99      2063
weighted avg       0.99      0.99      0.99      2063

